<a href="https://colab.research.google.com/github/dawikrol/Engineering-thesis/blob/main/Scripts/Wind_potential.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Wind potential**





### Lib import

In [25]:
import pandas as pd
from google.colab import files
import io
import os
import numpy as np

### Uploading data

You find data in project's GitHub repository in Data/Wind.

Data regarding atmospheric conditions come from: https://ec.europa.eu/jrc/en/pvgis



In [2]:
uploaded = files.upload()

Saving Limanowski_01_W.csv to Limanowski_01_W.csv
Saving Nowosądecki_01_W.csv to Nowosądecki_01_W.csv
Saving Nowotarski_01_W.csv to Nowotarski_01_W.csv
Saving NowySącz_01_W.csv to NowySącz_01_W.csv
Saving Tatrzański_01_W.csv to Tatrzański_01_W.csv


### Creating raw dataframes

In [3]:
dataframes = {}

for file in uploaded:
    name, region, kind  = file.split('_')
    dataframes[name] = pd.read_csv(io.BytesIO(uploaded[file]), error_bad_lines=False,
                                   index_col=None, header=None, sep='\n')

dataframes[list(dataframes.keys())[0]].size

8787

In [4]:
dataframes[list(dataframes.keys())[0]].head(20)

,0
0,Latitude (decimal degrees): 49.723
1,Longitude (decimal degrees): 20.395
2,Elevation (m): 400.0
3,"month,year"
4,"1,2006"
5,"2,2010"
6,"3,2006"
7,"4,2012"
8,"5,2012"
9,"6,2011"


In [5]:
dataframes[list(dataframes.keys())[0]].tail(20)

,0
8767,"20101231:1400,-3.55,84.14,19.0,0.0,19.0,276.6,..."
8768,"20101231:1500,-3.31,85.77,0.0,-0.0,0.0,276.12,..."
8769,"20101231:1600,-3.28,89.09,0.0,-0.0,0.0,269.75,..."
8770,"20101231:1700,-3.24,89.12,0.0,-0.0,0.0,269.16,..."
8771,"20101231:1800,-3.2,89.15,0.0,-0.0,0.0,268.57,3..."
8772,"20101231:1900,-3.16,89.18,0.0,-0.0,0.0,267.98,..."
8773,"20101231:2000,-3.12,89.22,0.0,-0.0,0.0,267.39,..."
8774,"20101231:2100,-3.08,89.25,0.0,-0.0,0.0,266.8,3..."
8775,"20101231:2200,-3.04,89.28,0.0,-0.0,0.0,266.21,..."
8776,"20101231:2300,-3.0,89.31,0.0,-0.0,0.0,265.62,3..."


### List of shortcuts

In [6]:
info_df = dataframes[list(dataframes.keys())[0]].iloc[8777:8786]
info_df.columns = ['defu']
info_df = info_df.defu.str.split(':', expand=True)
info_df.columns = ['symbol', 'meaning']
info_df.set_index('symbol', inplace=True)
info_df


,meaning
symbol,
T2m,2-m air temperature (degree Celsius)
RH,relative humidity (%)
G(h),Global irradiance on the horizontal plane (W/m2)
Gb(n),Beam/direct irradiance on a plane always norm...
Gd(h),Diffuse irradiance on the horizontal plane (W...
IR(h),Surface infrared (thermal) irradiance on a ho...
WS10m,10-m total wind speed (m/s)
WD10m,"10-m wind direction (0 = N, 90 = E) (degree)"
SP,Surface (air) pressure (Pa)


### Data cleaning

In [9]:
cleared_df = {}

for key in dataframes:
    cl_data = dataframes[key].iloc[16:8777]
    cl_data.columns = ['defu'] #the default column name is inconvenient
    cl_data = cl_data.defu.str.split(',', expand=True)
    cl_data.columns = cl_data.iloc[0]
    cl_data = cl_data.reset_index(drop=True)
    cl_data = cl_data.drop(cl_data.index[0])
    cl_data.columns.name = None
    #cl_data = cl_data.reset_index(drop=True)
    cl_data['Day'] = np.arange(1, len(cl_data)+1) #adding 'Day' maybe confused here because is similar to index but it will be really helpful latter
    cl_data['time(UTC)'] = pd.to_datetime(cl_data['time(UTC)'], format='%Y%m%d:%H%M')

    cleared_df[key] = cl_data

cleared_df[list(dataframes.keys())[0]]

,time(UTC),T2m,RH,G(h),Gb(n),Gd(h),IR(h),WS10m,WD10m,SP,Day
1,2006-01-01 00:00:00,-2.96,89.35,0.0,-0.0,0.0,265.03,4.07,190.0,96549.0,1
2,2006-01-01 01:00:00,-2.92,89.38,0.0,-0.0,0.0,264.43,4.19,184.0,96549.0,2
3,2006-01-01 02:00:00,-2.88,89.41,0.0,-0.0,0.0,263.84,4.31,178.0,96549.0,3
4,2006-01-01 03:00:00,-2.84,89.44,0.0,-0.0,0.0,263.25,4.44,176.0,96536.0,4
5,2006-01-01 04:00:00,-2.8,89.48,0.0,-0.0,0.0,262.66,4.56,175.0,96524.0,5
...,...,...,...,...,...,...,...,...,...,...,...
8756,2010-12-31 19:00:00,-3.16,89.18,0.0,-0.0,0.0,267.98,3.45,264.0,97325.0,8756
8757,2010-12-31 20:00:00,-3.12,89.22,0.0,-0.0,0.0,267.39,3.58,264.0,97321.0,8757
8758,2010-12-31 21:00:00,-3.08,89.25,0.0,-0.0,0.0,266.8,3.7,261.0,97293.0,8758
8759,2010-12-31 22:00:00,-3.04,89.28,0.0,-0.0,0.0,266.21,3.82,257.0,97264.0,8759


###Upload data regarding wind turbins

In [14]:
uploaded_turbins = files.upload()

Saving Turbins.xlsx to Turbins (1).xlsx


In [15]:
for file in uploaded_turbins:
    turbine_2MW = pd.read_excel(io.BytesIO(uploaded_turbins[file]), sheet_name='2MW')
    turbine_20kW = pd.read_excel(io.BytesIO(uploaded_turbins[file]), sheet_name='20kW')

turbine_2MW.columns = ['WS80m', 'P80m']
turbine_2MW.drop(turbine_2MW.index[0], inplace=True)
turbine_2MW = turbine_2MW.astype(float)

turbine_20kW.columns = ['WS30m', 'P30m']
turbine_20kW.drop(turbine_20kW.index[0], inplace=True)
turbine_20kW = turbine_20kW.astype(float)

In [ ]:
turbine_2MW.head(10)

,WS80m,P80m
1,0.0,0.0
2,0.5,0.0
3,1.0,0.0
4,1.5,0.0
5,2.0,0.0
6,2.5,0.0
7,3.0,0.0
8,3.5,42.2
9,4.0,93.3
10,4.5,145.2


In [ ]:
turbine_20kW.head(10)

,WS30m,P30m
1,0.0,0.00
2,1.0,0.00
3,2.0,0.08
4,3.0,0.50
5,4.0,1.35
6,5.0,2.80
7,6.0,4.70
8,7.0,7.00
9,8.0,9.60
10,9.0,12.30


###Calculations


1. Assignment of variable values


    aplpha - roughness coefficient (exponent)

    h_orginal - wind measurement high [m]

    h_t1 - height of the first wind turbin [m]

    h_t2 - height of the second wind turbin [m]


In [16]:

alpha = 0.16
h_orginal = 10
h_t1 = 30
h_t2 = 80



2. Calculation of wind speed at turbine heights

In [17]:
pd.options.mode.chained_assignment = None  # default='warn', pandas is warning that my operations are not apply on my orginal frame. It's true and it's what I want here

final_results = {}

for key in cleared_df:
    cl_data = cleared_df[key]
    cl_data['WS10m'] = cl_data['WS10m'].astype(float)
    cl_data['WS30m'] = cl_data['WS10m'].apply(lambda x: x*(h_t1/h_orginal)**(alpha))
    cl_data['WS80m'] = cl_data['WS10m'].apply(lambda x: x*(h_t2/h_orginal)**(alpha))
    final_results[key] = cl_data
final_results[list(dataframes.keys())[0]]

,time(UTC),T2m,RH,G(h),Gb(n),Gd(h),IR(h),WS10m,WD10m,SP,Day,WS30m,WS80m
1,2006-01-01 00:00:00,-2.96,89.35,0.0,-0.0,0.0,265.03,4.07,190.0,96549.0,1,4.852145,5.676607
2,2006-01-01 01:00:00,-2.92,89.38,0.0,-0.0,0.0,264.43,4.19,184.0,96549.0,2,4.995206,5.843976
3,2006-01-01 02:00:00,-2.88,89.41,0.0,-0.0,0.0,263.84,4.31,178.0,96549.0,3,5.138267,6.011345
4,2006-01-01 03:00:00,-2.84,89.44,0.0,-0.0,0.0,263.25,4.44,176.0,96536.0,4,5.293250,6.192662
5,2006-01-01 04:00:00,-2.8,89.48,0.0,-0.0,0.0,262.66,4.56,175.0,96524.0,5,5.436310,6.360031
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8756,2010-12-31 19:00:00,-3.16,89.18,0.0,-0.0,0.0,267.98,3.45,264.0,97325.0,8756,4.112998,4.811866
8757,2010-12-31 20:00:00,-3.12,89.22,0.0,-0.0,0.0,267.39,3.58,264.0,97321.0,8757,4.267981,4.993182
8758,2010-12-31 21:00:00,-3.08,89.25,0.0,-0.0,0.0,266.8,3.70,261.0,97293.0,8758,4.411041,5.160552
8759,2010-12-31 22:00:00,-3.04,89.28,0.0,-0.0,0.0,266.21,3.82,257.0,97264.0,8759,4.554102,5.327921


3. Matching the turbine power value depending on the wind speed

In [18]:
for key in final_results:

    dataP30 = pd.merge_asof(final_results[key].sort_values('WS30m'), turbine_20kW, on='WS30m')
    final_results[key] = dataP30
    dataP80 = pd.merge_asof(final_results[key].sort_values('WS80m'), turbine_2MW, on='WS80m')
    dataP80.sort_values('Day', inplace=True) 
    dataP80.set_index('Day', inplace=True)
    final_results[key] = dataP80
    
final_results[list(dataframes.keys())[0]]

,time(UTC),T2m,RH,G(h),Gb(n),Gd(h),IR(h),WS10m,WD10m,SP,WS30m,WS80m,P30m,P80m
Day,,,,,,,,,,,,,,
1,2006-01-01 00:00:00,-2.96,89.35,0.0,-0.0,0.0,265.03,4.07,190.0,96549.0,4.852145,5.676607,1.35,284.2
2,2006-01-01 01:00:00,-2.92,89.38,0.0,-0.0,0.0,264.43,4.19,184.0,96549.0,4.995206,5.843976,1.35,284.2
3,2006-01-01 02:00:00,-2.88,89.41,0.0,-0.0,0.0,263.84,4.31,178.0,96549.0,5.138267,6.011345,2.80,390.9
4,2006-01-01 03:00:00,-2.84,89.44,0.0,-0.0,0.0,263.25,4.44,176.0,96536.0,5.293250,6.192662,2.80,390.9
5,2006-01-01 04:00:00,-2.8,89.48,0.0,-0.0,0.0,262.66,4.56,175.0,96524.0,5.436310,6.360031,2.80,390.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8756,2010-12-31 19:00:00,-3.16,89.18,0.0,-0.0,0.0,267.98,3.45,264.0,97325.0,4.112998,4.811866,1.35,145.2
8757,2010-12-31 20:00:00,-3.12,89.22,0.0,-0.0,0.0,267.39,3.58,264.0,97321.0,4.267981,4.993182,1.35,145.2
8758,2010-12-31 21:00:00,-3.08,89.25,0.0,-0.0,0.0,266.8,3.70,261.0,97293.0,4.411041,5.160552,1.35,211.3


### Calculating average of yearly electricity production for all region

In [19]:
num_regions = len(final_results)
sum_energy_30=0
sum_energy_80=0 

for key in final_results:
    sum_energy_30 += final_results[key]['P30m'].sum()
    sum_energy_80 += final_results[key]['P80m'].sum()

ave_30 = sum_energy_30/num_regions
ave_80 = sum_energy_80/num_regions

Average amount of energy produced annually by a 20kW turbine for the region [GWh]: 

In [50]:
ave_30_GWh = np.round(ave_30/1000000, 4)
ave_30_GWh

0.0102

Average amount of energy produced annually by a 2MW turbine for the region [GWh]:

In [52]:
ave_80_GWh = np.round(ave_80/1000000, 4)
ave_80_GWh

1.4717

### Saving to files


In [ ]:
from google.colab import files

In [ ]:
for key in final_results:
    final_results[key].to_excel(f'{key}_w_results.xlsx')
    files.download(f'{key}_w_results.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Saving to database

In [87]:
!sudo apt-get install python3-dev default-libmysqlclient-dev
!pip install pymysql
!pip install SQLAlchemy

Reading package lists... Done
Building dependency tree       
Reading state information... Done
default-libmysqlclient-dev is already the newest version (1.0.4).
python3-dev is already the newest version (3.6.7-1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [99]:
import os
from sqlalchemy import create_engine

MYSQL_HOSTNAME = 'localhost'
MYSQL_USER = os.environ.get('DB_USER')
MYSQL_PASSWORD = os.environ.get('DB_PASS')
MYSQL_DATABASE = 'engineering_data'

connection_string = f'mysql+pymysql://{MYSQL_USER}:{MYSQL_PASSWORD}@{MYSQL_HOSTNAME}/{MYSQL_DATABASE}'
engine = create_engine(connection_string)


In [102]:
for key in final_results:
    final_results[key].to_sql(f'{key}', engine)